In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\Dede
[nltk_data]     Avriana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## LoAD dataset

In [3]:
data = pd.read_excel('DATASET CYBERBULLYING INSTAGRAM - FINAL.xlsx')
data.head()

,No.,Nama Instagram,Komentar,Kategori,Tanggal Posting,Nama Akun IG Artis/Selebgram,Unnamed: 6,Unnamed: 7
0,1,@delliananda,"""Kaka tidur yaa, udah pagi, gaboleh capek2""",Non-bullying,14 Oktober 2019,@isyanasarasvati,NaN,NaN
1,2,@fenninbl,"""makan nasi padang aja begini badannya""",Non-bullying,14 Oktober 2019,@isyanasarasvati,NaN,NaN
2,3,@abdurahmanshq,"""yang aku suka dari dia adalah selalu cukur je...",Bullying,14 Oktober 2019,@isyanasarasvati,NaN,NaN
3,4,@najla.yoo,"""Hai kak Isyana aku ngefans banget sama kak Is...",Non-bullying,14 Oktober 2019,@isyanasarasvati,NaN,NaN
4,5,@dessy_______,"""Manusia apa bidadari sih herann deh cantik te...",Non-bullying,14 Oktober 2019,@isyanasarasvati,NaN,NaN


## DROP DATA

In [4]:
data = data.drop (['No.','Nama Instagram','Tanggal Posting','Unnamed: 6','Nama Akun IG  Artis/Selebgram','Unnamed: 7'], axis=1)
data

,Komentar,Kategori
0,"""Kaka tidur yaa, udah pagi, gaboleh capek2""",Non-bullying
1,"""makan nasi padang aja begini badannya""",Non-bullying
2,"""yang aku suka dari dia adalah selalu cukur je...",Bullying
3,"""Hai kak Isyana aku ngefans banget sama kak Is...",Non-bullying
4,"""Manusia apa bidadari sih herann deh cantik te...",Non-bullying
...,...,...
645,"""aku memutuskan untuk menjadi fans isyana. gil...",Non-bullying
646,"""AMZING ISYANAA!! Jujur aku amazed banget deng...",Non-bullying
647,"""paling ngiri liat orang keren maen alat musik...",Non-bullying
648,"""Sampe ga bisa berkata2 lagi buat isyana, sang...",Non-bullying


## normalisasi label data

In [5]:
data.Kategori.replace("Bullying",0,inplace=True)
data.Kategori.replace("Non-bullying",1,inplace=True)
data.head(10)

,Komentar,Kategori
0,"""Kaka tidur yaa, udah pagi, gaboleh capek2""",1
1,"""makan nasi padang aja begini badannya""",1
2,"""yang aku suka dari dia adalah selalu cukur je...",0
3,"""Hai kak Isyana aku ngefans banget sama kak Is...",1
4,"""Manusia apa bidadari sih herann deh cantik te...",1
5,"""@ayu.kinantii isyan skrg berubah ya:( baju ny...",0
6,"""Gemesnya isyan kayak tango, berlapis lapis ci...",1
7,"""Makin jelek aja anaknya, padahal ibu ayahnya ...",0
8,"""Kok anaknya kayak udah tua gitu ya mukanya kk...",0
9,"""Muka anak nya ko tua banget yaa.. GK ngegemes...",0


## TEXT PROCESSING
## CASE FOLDING

In [6]:
import re

def casefolding(text):
    text = text.lower()
    text = re.sub(r'[😌\♥️\_\:)\:(]','',text)
    text = re.sub(r'[-+]?[0-9]+','',text)
    text = re.sub(r'[^\w\s]','',text)
    text = text.strip()
    return text


In [7]:
raw_sample = data['Komentar'].iloc[12]
case_folding =casefolding(raw_sample)

print('rawdata\t :',raw_sample)
print('case folding\t :', case_folding)

rawdata	 : "Lo sintingnya udah tingkat dewa"
case folding	 : lo sintingnya udah tingkat dewa


## word normalization

In [8]:
key_norm = pd.read_csv('kamus_singkatan.csv')
key_norm = key_norm.drop(['Unnamed: 0'],axis=1)

In [9]:
def text_normalize(text):
    text = ' '.join([key_norm[key_norm['singkatan'] == word]['asli'].values[0]
    if (key_norm['singkatan']==word).any()
    else word for word in text.split()
    ])

    text = str.lower(text)
    return text

In [10]:
#bandingkan
raw_data = data['Komentar'].iloc[578]
word_normal = text_normalize(raw_data)

print('rawdata\t :', raw_data)
print('Word Normalize\t :' , word_normal)

rawdata	 : "Terima kasih kak Nagita @raffinagita1717 sudah support Molly si gajah dan satwa-satwa di Bali Zoo lainnya dengan kunjungannya bersama keluarga"
Word Normalize	 : "terima kasih kak nagita @raffinagita1717 sudah support molly si gajah dan satwa-satwa di bali zoo lainnya dengan kunjungannya bersama keluarga"


## filter

In [11]:
from nltk.tokenize import sent_tokenize , word_tokenize
from nltk.corpus import stopwords

stopwords_ind = stopwords.words('indonesian')

In [12]:
len(stopwords_ind)

758

In [13]:
more_stopword = ['@rosameldianti','@ayu.kinantii', '@abatartila','@raffinagita1717','@syifahadjureal','@isyanasarasvati','@vnagstn','@jeromepoline','@maudyayunda']
stopwords_ind = stopwords_ind + more_stopword

def remove_stop_word(text):
    clean_word = []
    text = text.split()
    for word in text:
        if word not in stopwords_ind:
            clean_word.append(word)
    return " ".join(clean_word)

## stemming

In [14]:
!pip -q install sastrawi

In [15]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    text = stemmer.stem(text)
    return text

## pipeline

In [16]:
def text_preprocessing_process(text):
    text = casefolding(text)
    text = text_normalize(text)
    text = remove_stop_word(text)
    text = stemming(text)
    return text

In [17]:
%%time
data['clean_teks']= data['Komentar'].apply(text_preprocessing_process)

Wall time: 1min 41s


In [18]:
data

,Komentar,Kategori,clean_teks
0,"""Kaka tidur yaa, udah pagi, gaboleh capek2""",1,kaka tidur yaa pagi capai
1,"""makan nasi padang aja begini badannya""",1,makan nasi padang badan
2,"""yang aku suka dari dia adalah selalu cukur je...",0,suka cukur jembut manggung
3,"""Hai kak Isyana aku ngefans banget sama kak Is...",1,hai kak isyana ngefans kak isyanaaku suka lagu...
4,"""Manusia apa bidadari sih herann deh cantik te...",1,manusia bidadari sih herann deh cantik
...,...,...,...
645,"""aku memutuskan untuk menjadi fans isyana. gil...",1,putus fans isyana gila keren ya otak dua fungs...
646,"""AMZING ISYANAA!! Jujur aku amazed banget deng...",1,amzing isyanaa jujur amazed skill minat isyana...
647,"""paling ngiri liat orang keren maen alat musik...",1,ngiri liat orang keren main alat musik nikmati...
648,"""Sampe ga bisa berkata2 lagi buat isyana, sang...",1,isyana sempurna orang


In [19]:
data.to_csv('clean_data.csv')

## feature enginering

In [20]:
x = data['clean_teks']
y = data['Kategori']


In [21]:
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer

vec_TF_IDF =TfidfVectorizer(ngram_range=(1,1))
vec_TF_IDF.fit(x)

x_tf_idf = vec_TF_IDF.transform(x)

pickle.dump(vec_TF_IDF.vocabulary_,open("feature_tf-idf.sav","wb"))

In [22]:
vec_TF_IDF.vocabulary_

{'kaka': 647,
 'tidur': 1428,
 'yaa': 1537,
 'pagi': 1098,
 'capai': 274,
 'makan': 852,
 'nasi': 972,
 'padang': 1097,
 'badan': 121,
 'suka': 1366,
 'cukur': 313,
 'jembut': 615,
 'manggung': 868,
 'hai': 502,
 'kak': 646,
 'isyana': 587,
 'ngefans': 999,
 'isyanaaku': 589,
 'lagu': 777,
 'dalam': 325,
 'jiwa': 630,
 'manusia': 875,
 'bidadari': 204,
 'sih': 1314,
 'herann': 533,
 'deh': 336,
 'cantik': 269,
 'ayukinantii': 110,
 'isyan': 586,
 'ubah': 1471,
 'ya': 1536,
 'baju': 132,
 'nya': 1057,
 'nakal': 962,
 'gemesnya': 456,
 'tango': 1399,
 'lap': 789,
 'lapis': 791,
 'ciaaaa': 288,
 'jelek': 613,
 'anak': 57,
 'ayah': 107,
 'cakap': 264,
 'tua': 1454,
 'muka': 941,
 'kartu': 672,
 'keluarga': 698,
 'tasya': 1404,
 'ko': 731,
 'ngegemesin': 1000,
 'lucu': 831,
 'kolot': 732,
 'wkwk': 1524,
 'bayi': 169,
 'gila': 466,
 'orang': 1087,
 'jomblo': 632,
 'psikolog': 1179,
 'laku': 780,
 'sinting': 1326,
 'tingkat': 1435,
 'dewa': 343,
 'lutfi': 835,
 'putus': 1189,
 'salsa': 1239,


In [23]:
x1 = vec_TF_IDF.transform(x).toarray()
data_tabular_tf_idf = pd.DataFrame(x1,columns=vec_TF_IDF.get_feature_names())
data_tabular_tf_idf

c:\Users\Dede Avriana\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,aaa,aamiin,abai,abalabal,abatartila,abu,activity,ada,adab,...,yemuke,yesus,you,younger,youtube,youtuber,yuhuuu,yutuber,ziu,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Feature Selection

In [24]:
x_train =np.array(data_tabular_tf_idf)
y_train= np.array(y)

In [26]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

chi2_features = SelectKBest(chi2, k=1500)
x_best_features =chi2_features.fit_transform(x_train,y_train)

print('Original Feature Number', x_train.shape[1])
print('Reduced Feature Number', x_best_features.shape[1])

Original Feature Number 1561
Reduced Feature Number 1500


In [27]:
Data = pd.DataFrame(chi2_features.scores_,columns=['Nilai'])
Data

,Nilai
0,0.267741
1,0.496982
2,1.730319
3,0.483121
4,0.388735
...,...
1556,0.389909
1557,0.366535
1558,0.372156
1559,0.445437


In [28]:
feature = vec_TF_IDF.get_feature_names()
feature

Data['Fitur']= feature
Data

c:\Users\Dede Avriana\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Nilai,Fitur
0,0.267741,aa
1,0.496982,aaa
2,1.730319,aamiin
3,0.483121,abai
4,0.388735,abalabal
...,...,...
1556,0.389909,youtuber
1557,0.366535,yuhuuu
1558,0.372156,yutuber
1559,0.445437,ziu


In [29]:
Data.sort_values(by='Nilai',ascending=False)

,Nilai,Fitur
68,1.832014e+01,anjing
269,1.118953e+01,cantik
941,1.009703e+01,muka
114,7.971411e+00,babi
646,7.773035e+00,kak
...,...,...
1528,3.084535e-04,wkwkwkwk
1555,3.066811e-04,youtube
107,1.989175e-04,ayah
275,1.032682e-04,caption


In [30]:
mask = chi2_features.get_support()
mask

array([ True,  True,  True, ...,  True,  True,  True])

In [31]:
new_feature=[]
for bool, f in zip(mask,feature):
    if bool :
        new_feature.append(f)
    selected_feature=new_feature
selected_feature

['aa',
 'aaa',
 'aamiin',
 'abai',
 'abalabal',
 'abatartila',
 'activity',
 'ada',
 'adab',
 'adam',
 'adaptasi',
 'adeknya',
 'adi',
 'adik',
 'aduh',
 'aduhh',
 'aduuh',
 'aesthetic',
 'agama',
 'age',
 'ager',
 'ah',
 'ahahah',
 'ahhaha',
 'aih',
 'aing',
 'air',
 'ajar',
 'ajg',
 'ajgg',
 'aji',
 'aka',
 'akhlak',
 'akhlakngajarin',
 'aki',
 'aktif',
 'akun',
 'alam',
 'alas',
 'alat',
 'alay',
 'alisny',
 'allah',
 'alus',
 'always',
 'amatt',
 'amazed',
 'ambil',
 'amen',
 'amin',
 'amplas',
 'ampun',
 'amzing',
 'an',
 'anak',
 'and',
 'andy',
 'angelkmgi',
 'anggun',
 'angkut',
 'anj',
 'anjay',
 'anjeng',
 'anjg',
 'anjim',
 'anjing',
 'anjingeric',
 'anjir',
 'anjrit',
 'anjso',
 'anteng',
 'aowkwkw',
 'apa',
 'apaa',
 'apalo',
 'apartemen',
 'apasih',
 'apresiasi',
 'are',
 'ariana',
 'aroma',
 'artis',
 'arwah',
 'asa',
 'asing',
 'asli',
 'astaga',
 'astagah',
 'astaghfirullah',
 'astral',
 'asu',
 'asuh',
 'asuuu',
 'asyik',
 'at',
 'atas',
 'atik',
 'atm',
 'atuh',
 'at

In [32]:
new_selected_feature = {}
for (k,v) in vec_TF_IDF.vocabulary_.items():
    if k in selected_feature:
        new_selected_feature[k]=v
new_selected_feature

{'kaka': 647,
 'tidur': 1428,
 'yaa': 1537,
 'pagi': 1098,
 'capai': 274,
 'makan': 852,
 'nasi': 972,
 'padang': 1097,
 'badan': 121,
 'suka': 1366,
 'cukur': 313,
 'jembut': 615,
 'manggung': 868,
 'hai': 502,
 'kak': 646,
 'isyana': 587,
 'ngefans': 999,
 'isyanaaku': 589,
 'lagu': 777,
 'dalam': 325,
 'manusia': 875,
 'bidadari': 204,
 'sih': 1314,
 'herann': 533,
 'cantik': 269,
 'ayukinantii': 110,
 'isyan': 586,
 'ubah': 1471,
 'ya': 1536,
 'baju': 132,
 'nya': 1057,
 'nakal': 962,
 'gemesnya': 456,
 'tango': 1399,
 'lap': 789,
 'lapis': 791,
 'ciaaaa': 288,
 'jelek': 613,
 'anak': 57,
 'cakap': 264,
 'tua': 1454,
 'muka': 941,
 'tasya': 1404,
 'ko': 731,
 'kolot': 732,
 'wkwk': 1524,
 'bayi': 169,
 'gila': 466,
 'jomblo': 632,
 'psikolog': 1179,
 'laku': 780,
 'sinting': 1326,
 'tingkat': 1435,
 'dewa': 343,
 'lutfi': 835,
 'putus': 1189,
 'salsa': 1239,
 'stress': 1356,
 'tolol': 1444,
 'aktif': 36,
 'mati': 890,
 'gedeg': 448,
 'liat': 810,
 'anjim': 67,
 'woy': 1531,
 'asu':

In [34]:
len(new_selected_feature)

1500

In [35]:
pickle.dump(new_selected_feature,open("new_selected_feature_tf-idf.sav","wb"))


In [36]:
data_selected_feature = pd.DataFrame(x_best_features, columns=selected_feature)
data_selected_feature

,aa,aaa,aamiin,abai,abalabal,abatartila,activity,ada,adab,adam,...,yehh,yemuke,yesus,you,younger,youtuber,yuhuuu,yutuber,ziu,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## MOdeling

In [37]:
selected_x = x_best_features
selected_x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [42]:
x = selected_x
y = data.Kategori

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [43]:
print('x_train : ',len(x_train))
print('x_test : ',len(x_test))
print('y_train : ',len(y_train))
print('y_test : ',len(y_test))

x_train :  520
x_test :  130
y_train :  520
y_test :  130


In [44]:
text_algorithm = MultinomialNB()

In [45]:
model = text_algorithm.fit(x_train,y_train)


In [47]:
data_input = ("paling ngiri liat orang keren maen alat musik dan dalem banget nikmatin permainan sendiri")
data_input = text_preprocessing_process(data_input)

tfidf = TfidfVectorizer
loaded_vec = TfidfVectorizer(decode_error="replace",vocabulary=set(pickle.load(open("new_selected_feature_tf-idf.sav","rb"))))
hasil = model.predict(loaded_vec.fit_transform([data_input]))

if (hasil==1):
    s = "Komentar bersifat Non Bullying"
else:
    s = "Komentar bersifat Bullying"

print("Hasil Prediksi : \n", s) 

Hasil Prediksi : 
 Komentar Non Bullying


## Evaluasi MOdel

In [48]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

predicted = model.predict(x_test)
CM = confusion_matrix(y_test,predicted)

print(classification_report(y_test,predicted))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86        66
           1       0.88      0.81      0.85        64

    accuracy                           0.85       130
   macro avg       0.86      0.85      0.85       130
weighted avg       0.86      0.85      0.85       130



In [50]:
pickle.dump(model,open("model_ciberbully.sav","wb"))